## CH8

### 8.1

In [1]:
class Pair:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __repr__(self):
        return 'Pair({0.x!r}, {0.y!r})'.format(self)
    def __str__(self):
        return '({0.x!s}, {0.y!s})'.format(self)

In [2]:
p = Pair(3, 4)

In [3]:
p

Pair(3, 4)

In [4]:
print(p)

(3, 4)


In [5]:
print('p is {0!r}'.format(p))   ## 默認__repr__()

p is Pair(3, 4)


In [6]:
print('p is {0}'.format(p))

p is (3, 4)


### 8.2

In [7]:
_formats = {
    'ymd' : '{d.year}-{d.month}-{d.day}',
    'mdy' : '{d.month}/{d.day}/{d.year}',
    'dmy' : '{d.day}/{d.month}/{d.year}'
    }

class Date:
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day

    def __format__(self, code):
        if code == '':
            code = 'ymd'
        fmt = _formats[code]
        return fmt.format(d=self)

In [8]:
d = Date(2012, 12, 21)

In [9]:
format(d)   ## __format__()

'2012-12-21'

In [10]:
format(d, 'mdy')

'12/21/2012'

In [11]:
'The date is {:ymd}'.format(d)

'The date is 2012-12-21'

In [12]:
'The date is {:mdy}'.format(d)

'The date is 12/21/2012'

In [13]:
from datetime import date
d = date(2012, 12, 21)

In [14]:
format(d)

'2012-12-21'

In [15]:
format(d,'%A, %B %d, %Y')

'Friday, December 21, 2012'

In [16]:
'Friday, December 21, 2012'

'Friday, December 21, 2012'

### 8.3

In [17]:
from socket import socket, AF_INET, SOCK_STREAM

class LazyConnection:
    def __init__(self, address, family=AF_INET, type=SOCK_STREAM):
        self.address = address
        self.family = family
        self.type = type
        self.sock = None

    def __enter__(self):
        if self.sock is not None:
            raise RuntimeError('Already connected')
        self.sock = socket(self.family, self.type)
        self.sock.connect(self.address)
        return self.sock

    def __exit__(self, exc_ty, exc_val, tb):
        self.sock.close()
        self.sock = None

In [18]:
from functools import partial

conn = LazyConnection(('www.python.org', 80))
# Connection closed
with conn as s:
    # conn.__enter__() executes: connection open
    s.send(b'GET /index.html HTTP/1.0\r\n')
    s.send(b'Host: www.python.org\r\n')
    s.send(b'\r\n')
    resp = b''.join(iter(partial(s.recv, 8192), b''))
    # conn.__exit__() executes: connection closed

* enter返回的值(如果有的話)會被賦值給as聲明的變量
* 不管with中發生什麼，控制流都會執行完
* 上面的定義中一次只能允許一個socket連接，如果正在使用一個socket的時候又重複使用with 語句，就會產生一個異常，修改如下

In [19]:
from socket import socket, AF_INET, SOCK_STREAM

class LazyConnection:
    def __init__(self, address, family=AF_INET, type=SOCK_STREAM):
        self.address = address
        self.family = family
        self.type = type
        self.connections = []

    def __enter__(self):
        sock = socket(self.family, self.type)
        sock.connect(self.address)
        self.connections.append(sock)
        return sock

    def __exit__(self, exc_ty, exc_val, tb):
        self.connections.pop().close()

# Example use
from functools import partial

conn = LazyConnection(('www.python.org', 80))
with conn as s1:
    pass
    with conn as s2:
        pass
        # s1 and s2 are independent sockets

### 8.4

* 對於主要是用來當成簡單的數據結構的類而言，可通過給類添加slots屬性來減少內存
* Python的很多特性都依賴於普通的基於字典的實現，定義了slots後的類不再支持一些普通類特性了，比如多繼承

In [20]:
class Date:
    __slots__ = ['year', 'month', 'day']
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day

### 8.5

* Python並不會真的阻止別人訪問內部名稱，但是如果你這麼做肯定是不好的，可能會導致脆弱的代碼
* 內部實現

In [21]:
class A:
    def __init__(self):
        self._internal = 0 # An internal attribute
        self.public = 1 # A public attribute

    def public_method(self):
        '''
        A public method
        '''
        pass

    def _internal_method(self):
        pass

* 使用雙下劃線開始會導致訪問名稱變成其他形式
* 類B中，私有屬性會被分別重命名爲\_B_\_private 和 \_B_\_private_method 
* 重命名的目的是什麼?答案就是繼承——這種屬性通過繼承是無法被覆蓋的

In [22]:
class B:
    def __init__(self):
        self.__private = 0

    def __private_method(self):
        pass

    def public_method(self):
        pass
        self.__private_method()

In [23]:
class C(B):
    def __init__(self):
        super().__init__()
        self.__private = 1 # Does not override B.__private

    # Does not override B.__private_method()
    def __private_method(self):
        pass

In [24]:
lambda_ = 2.0 # Trailing _ to avoid clash with lambda keyword

### 8.6

* property:自定義某個屬性

In [25]:
class Person:
    def __init__(self, first_name):
        self.first_name = first_name

    # Getter function
    @property
    def first_name(self):
        return self._first_name

    # Setter function
    @first_name.setter
    def first_name(self, value):
        if not isinstance(value, str):
            raise TypeError('Expected a string')
        self._first_name = value

    # Deleter function (optional)
    @first_name.deleter
    def first_name(self):
        raise AttributeError("Can't delete attribute")

In [26]:
a = Person('Guido')

In [27]:
a.first_name

'Guido'

In [28]:
a.first_name = 42

TypeError: Expected a string

In [29]:
del a.first_name

AttributeError: Can't delete attribute

In [30]:
class Person:
    def __init__(self, first_name):
        self.set_first_name(first_name)

    # Getter function
    def get_first_name(self):
        return self._first_name

    # Setter function
    def set_first_name(self, value):
        if not isinstance(value, str):
            raise TypeError('Expected a string')
        self._first_name = value

    # Deleter function (optional)
    def del_first_name(self):
        raise AttributeError("Can't delete attribute")

    # Make a property from existing get/set methods
    name = property(get_first_name, set_first_name, del_first_name)

In [31]:
Person.get_first_name    #講義有誤

<function __main__.Person.get_first_name>

In [32]:
Person.get_first_name.fget   #無法執行

AttributeError: 'function' object has no attribute 'fget'

* Property這類型的attributes並不會被實際的存儲，而是在需要的時候計算出來

In [33]:
import math
class Circle:
    def __init__(self, radius):
        self.radius = radius

    @property
    def area(self):
        return math.pi * self.radius ** 2

    @property
    def diameter(self):
        return self.radius * 2

    @property
    def perimeter(self):
        return 2 * math.pi * self.radius

In [34]:
c = Circle(4.0)

In [35]:
c.radius

4.0

In [36]:
p = Person('Guido')

In [37]:
p.get_first_name()

'Guido'

### 8.7

In [38]:
class A:
    def spam(self):
        print('A.spam')

class B(A):
    def spam(self):
        print('B.spam')
        super().spam()  # Call parent spam()

In [39]:
class A:
    def __init__(self):
        self.x = 0

class B(A):
    def __init__(self):
        super().__init__()
        self.y = 1

In [40]:
class Proxy:
    def __init__(self, obj):
        self._obj = obj

    # Delegate attribute lookup to internal obj
    def __getattr__(self, name):
        return getattr(self._obj, name)

    # Delegate attribute assignment
    def __setattr__(self, name, value):
        if name.startswith('_'):
            super().__setattr__(name, value) # Call original __setattr__
        else:
            setattr(self._obj, name, value)

In [41]:
class Base:
    def __init__(self):
        print('Base.__init__')

class A(Base):
    def __init__(self):
        Base.__init__(self)
        print('A.__init__')

In [42]:
class Base:
    def __init__(self):
        print('Base.__init__')

class A(Base):
    def __init__(self):
        Base.__init__(self)
        print('A.__init__')

class B(Base):
    def __init__(self):
        Base.__init__(self)
        print('B.__init__')

class C(A,B):
    def __init__(self):
        A.__init__(self)
        B.__init__(self)
        print('C.__init__')

In [43]:
c = C()

Base.__init__
A.__init__
Base.__init__
B.__init__
C.__init__


In [44]:
class Base:
    def __init__(self):
        print('Base.__init__')

class A(Base):
    def __init__(self):
        super().__init__()
        print('A.__init__')

class B(Base):
    def __init__(self):
        super().__init__()
        print('B.__init__')

class C(A,B):
    def __init__(self):
        super().__init__()  # Only one call to super() here
        print('C.__init__')

In [45]:
c = C()

Base.__init__
B.__init__
A.__init__
C.__init__


In [46]:
C.__mro__

(__main__.C, __main__.A, __main__.B, __main__.Base, object)

* 子類會先於父類被檢查
* 多個父類會根據它們在列表中的順序被檢查
* 如果對下一個類存在兩個合法的選擇，選擇第一個父類

* 當你使用super() 函數時，Python 會在MRO 列表上繼續搜索下一個類。只要每
個重定義的方法統一使用super() 並只調用它一次，那麼控制流最終會遍歷完整個
MRO 列表，每個方法也只會被調用一次。這也是爲什麼在第二個例子中你不會調用兩
次Base. init () 的原因。
super() 有個令人吃驚的地方是它並不一定去查找某個類在MRO 中下一個直接父
類，你甚至可以在一個沒有直接父類的類中使用它。例如，考慮如下這個類：

In [47]:
class A:
    def spam(self):
        print('A.spam')
        super().spam()

In [48]:
a = A()

In [49]:
a.spam()

A.spam


AttributeError: 'super' object has no attribute 'spam'

In [50]:
class B:
    def spam(self):
        print('B.spam')
        
class C(A,B):
    pass

In [51]:
c = C()

In [52]:
c.spam()

A.spam
B.spam


### 8.8

In [53]:
class Person:
    def __init__(self, name):
        self.name = name

    # Getter function
    @property
    def name(self):
        return self._name

    # Setter function
    @name.setter
    def name(self, value):
        if not isinstance(value, str):
            raise TypeError('Expected a string')
        self._name = value

    # Deleter function
    @name.deleter
    def name(self):
        raise AttributeError("Can't delete attribute")

In [54]:
class SubPerson(Person):
    @property
    def name(self):
        print('Getting name')
        return super().name

    @name.setter
    def name(self, value):
        print('Setting name to', value)
        super(SubPerson, SubPerson).name.__set__(self, value)

    @name.deleter
    def name(self):
        print('Deleting name')
        super(SubPerson, SubPerson).name.__delete__(self)